In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from datetime import datetime, timedelta

In [3]:
import pygrib
import numpy as np

In [4]:
import xarray as xr; print(xr.__version__)

0.14.0


In [5]:
import os

In [6]:
import pathlib

In [7]:
dpath = pathlib.Path('/home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/t2a/')

In [8]:
lfiles = list(dpath.glob("*.grb2"))

In [9]:
len(lfiles)

864

### loops over the files and process 

In [ ]:
for fname in lfiles: 
    
    print(f"processing {fname}")
    
    out_fname = fname.name.replace('.grb2','.nc')
    out_fname = fname.parent / out_fname
    init_date = datetime.strptime(fname.name.split('.')[-2], "%Y%m%d")
    
    grbs = pygrib.open(str(fname))
    
    analDates = []
    validDates = []
    validityDates = []
    data = []
    
    for g in grbs: 
        data.append(g.values)
        analDates.append(g.analDate)
        validDates.append(g.validDate)
        validityDates.append(datetime.strptime(str(g.validityDate), "%Y%m%d"))   
    
    data = np.array(data)

    data_reshaped = np.reshape(data, (len(np.unique(validDates)), data.shape[0] // len(np.unique(validDates)), data.shape[-2], data.shape[-1]))
    
    lats,lons = g.latlons()
    
    lats = lats[:,0]
    lons = lons[0,:]
    
    nmembers =  g.numberOfForecastsInEnsemble
    
    if nmembers != data_reshaped.shape[1]:
        print('oups, data shape doesnt match the number of ensemble members')
    
    d = {}

    d['forecast_valid_time'] = (('forecast_valid_time'), np.unique(validDates))
    d['latitude'] = (('latitude'), lats)
    d['longitude'] = (('longitude'), lons)
    d['number'] = (('number'), list(range(0, nmembers)))
    d[g.shortName.lower()] = (('forecast_valid_time', 'number', 'latitude','longitude'), data_reshaped)

    dset = xr.Dataset(d)

    ### mask the land values (np.nan)

    dset[g.shortName.lower()] = dset[g.shortName.lower()].where(dset[g.shortName.lower()] < 9999, np.nan)
    
    dset.to_netcdf(out_fname)
    
    dset.close()
    

processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/t2a/h2_Ptt_mon.19790116.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/t2a/h2_Ptt_mon.19790131.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/t2a/h2_Ptt_mon.19790210.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/t2a/h2_Ptt_mon.19790225.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/t2a/h2_Ptt_mon.19790312.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/t2a/h2_Ptt_mon.19790327.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/t2a/h2_Ptt_mon.19790411.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/t2a/h2_Ptt_mon.19790426.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/t2a/h2_Ptt_mon.197905